In [ ]:
import csv
import h5py
import numpy as np


traits = {'lat':[],'lon':[],'dsetid':[],'obs':[],'species':[],'45':[],'3122':[],
          '3120':[],'13':[],'146':[],'719':[],'6':[],'111':[],'63':[],'4':[],'flag':[]}
units = {'lat':'','lon':'','dsetid':'','obs':'','species':'','45':'','3122':'',
         '3120':'','13':'','146':'','719':'','6':'','111':'','63':'','4':'','flag':'0: clean, otherwise nonstd'}

names = {'lat':'lat','lon':'lon','dsetid':'dsetid','obs':'obs','species':'species',
         '45':'st_conductance','3122':'h20_sat','3120':'h20_nosat','13':'leafC',
         '146':'leafCN','719':'xylem','6':'root','111':'transpiration','63':'st_density','4':'SSD','flag':'flag'}
badobs = []
with open('7529_compressed.txt',encoding='utf-8') as csvfile:
    readCSV = csv.reader(csvfile, dialect="excel-tab")
    op = .5
    for row in readCSV:
        if((readCSV.line_num%38400) == 0):
            print(str(op)+'%')
            op = op + .5
        index = 0
        if (readCSV.line_num == 1):
            header = row
            continue
        if int(row[7]) in badobs:
            continue
        if (len(traits['obs'])==0):
            old_obs = False
        else:
            old_obs = (int(row[7]) == traits['obs'][len(traits['obs'])-1]) or (int(row[7]) in traits['obs'])
        
        if not old_obs:
            index = len(traits['obs'])
            for x in traits.keys():
                traits[x].append(-9999)
            traits['obs'][index]=int(row[7])
            traits['dsetid'][index]=int(row[2])
            traits['flag'][index] = np.string_('0')
            try:
                traits['species'][index]=np.string_((row[6])) 
            except:
                traits['species'][index]=np.string_((row[4])) 
        index = traits['obs'].index(int(row[7]))
        if (row[9]==''):
            if ((row[12] == 'Latitude') or (row[11]=='59')):
                try:
                    traits['lat'][index]=float(row[20])
                except:
                    try:
                        traits['lat'][index]=float(row[14])
                    except:
                        for var in traits.keys():
                            del traits[var][-1]
                        badobs.append(int(row[7]))
            elif ((row[12] == 'Longitude') or (row[11]=='60')):
                try:
                    traits['lon'][index]=float(row[20])
                except:
                    try:
                        traits['lon'][index]=float(row[14])
                    except:
                        for var in traits.keys():
                            del traits[var][-1]
                        badobs.append(int(row[7]))
        else:
            row9 = str(row[9])
            if row9 in traits.keys():
                #If it is xylem, search for the P50, represented by DataID = 1533,2099, or 6912
                if row9 == '719':
                    if (row[11]=='1533') or (row[11]=='2099') or (row[11]=='6912'):
                        try:
                            traits[row9][index] = float(row[14])
                            traits['flag'][index]= np.string_(row9+'_'+str(traits['obs'][index])+'_'+row[15].replace(" ",""))
                        except:
                            traits[row9][index] = -9999
                        newunit = np.string_(row[15].replace(" ",""))
                        if ((not ((units[row9]).lower()==(newunit).lower()) and (not (units[row9]=='')))):
                            traits['flag'][index]= np.string_(row9+'_'+str(traits['obs'][index])+'_'+row[15].replace(" ",""))
                        else:
                            units[row9] = np.string_(row[15].replace(" ",""))      
                else:
                    try:
                        traits[row9][index] = float(row[20])
                    except:
                        try:
                            traits[row9][index] = float(row[14])
                            traits['flag'][index]= np.string_(row9+'_'+str(traits['obs'][index])+'_'+row[15].replace(" ",""))
                        except:
                            traits[row9][index] = -9999
                    try:
                        if row[21] == "":
                            raise NameError("No STD Unit")
                        units[row9] = np.string_(row[21].replace(" ",""))
                    except:
                        newunit = np.string_(row[15].replace(" ",""))
                        if ((not ((units[row9]).lower()==(newunit).lower()) and (not (units[row9]=='')))):
                            traits['flag'][index]= np.string_(row9+'_'+str(traits['obs'][index])+'_'+row[15].replace(" ",""))
                        else:
                            units[row9] = np.string_(row[15].replace(" ",""))                                 
#n = len(obs)
print('Processing file...')
file = 'trydata_xcor.h5'
fp = h5py.File(file,'w')
for x in traits.keys():
    fp.create_dataset(names[x],data=np.array(traits[x]))
    fp[names[x]].attrs.create('units',np.string_(units[x]))
fp.close()
print('Complete')